In [46]:
!pip install scikeras

In [47]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from scikeras.wrappers import KerasRegressor
from sklearn.neural_network import MLPRegressor
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.utils.validation import check_is_fitted
from sklearn.exceptions import NotFittedError
from tensorflow.keras.callbacks import EarlyStopping
from xgboost import XGBRegressor
import numpy as np

In [48]:
import sklearn
print(sklearn.__version__)


1.6.1


In [49]:
import warnings
from sklearn.exceptions import FitFailedWarning

# Suppress specific sklearn warnings about NaN test scores
warnings.filterwarnings(
    action='ignore',
    message='One or more of the test scores are non-finite*',
    category=UserWarning
)

# Optional: suppress fit failures if you're trying risky hyperparameters
warnings.filterwarnings('ignore', category=FitFailedWarning)

In [50]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
excel_file_path = '/content/drive/MyDrive/HW3.xlsx'

df = pd.read_excel(excel_file_path)

print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   sequence_number  US  source_a  source_c  source_b  source_d  source_e  \
0                1   1         0         0         1         0         0   
1                2   1         0         0         0         0         1   
2                3   1         0         0         0         0         0   
3                4   1         0         1         0         0         0   
4                5   1         0         1         0         0         0   

   source_m  source_o  source_h  ...  source_x  source_w  Freq  \
0         0         0         0  ...         0         0     2   
1         0         0         0  ...         0         0     0   
2         0         0         0  ...         0         0     2   
3         0         0         0  ...         0         0     1   
4         0         0         0  ...         0         0     1   

   last_update_da

In [51]:
df

,sequence_number,US,source_a,source_c,source_b,source_d,source_e,source_m,source_o,source_h,...,source_x,source_w,Freq,last_update_days_ago,1st_update_days_ago,Web order,Gender=male,Address_is_res,Purchase,Spending
0,1,1,0,0,1,0,0,0,0,0,...,0,0,2,3662,3662,1,0,1,1,127.87
1,2,1,0,0,0,0,1,0,0,0,...,0,0,0,2900,2900,1,1,0,0,0.00
2,3,1,0,0,0,0,0,0,0,0,...,0,0,2,3883,3914,0,0,0,1,127.48
3,4,1,0,1,0,0,0,0,0,0,...,0,0,1,829,829,0,1,0,0,0.00
4,5,1,0,1,0,0,0,0,0,0,...,0,0,1,869,869,0,0,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,1,0,0,0,0,0,0,0,0,...,0,0,1,1701,1701,1,0,0,1,29.50
1996,1997,1,0,0,0,0,0,0,0,0,...,0,0,1,2633,2633,1,1,0,1,10.00
1997,1998,1,0,0,0,0,0,0,0,0,...,0,0,0,3394,3394,0,0,0,0,0.00
1998,1999,1,0,0,0,0,0,0,0,0,...,0,1,1,253,253,0,1,1,0,0.06


In [52]:
print(df.isnull().any())


sequence_number         False
US                      False
source_a                False
source_c                False
source_b                False
source_d                False
source_e                False
source_m                False
source_o                False
source_h                False
source_r                False
source_s                False
source_t                False
source_u                False
source_p                False
source_x                False
source_w                False
Freq                    False
last_update_days_ago    False
1st_update_days_ago     False
Web order               False
Gender=male             False
Address_is_res          False
Purchase                False
Spending                False
dtype: bool


### Q1) (A)

In [53]:
df.drop(['sequence_number'], axis=1, inplace=True)

In [54]:
from sklearn.model_selection import train_test_split

X_features = df.drop(columns=['Purchase', 'Spending'], axis=1)
y_feature = df['Spending']

In [55]:
X_CV, X_test, y_CV, y_test = train_test_split(
    X_features, y_feature, test_size=0.20, random_state=60
)

In [56]:
X_CV.columns

Index(['US', 'source_a', 'source_c', 'source_b', 'source_d', 'source_e',
       'source_m', 'source_o', 'source_h', 'source_r', 'source_s', 'source_t',
       'source_u', 'source_p', 'source_x', 'source_w', 'Freq',
       'last_update_days_ago', '1st_update_days_ago', 'Web order',
       'Gender=male', 'Address_is_res'],
      dtype='object')

In [57]:
binary_cols = ['US', 'source_a', 'source_c', 'source_b', 'source_d', 'source_e',
       'source_m', 'source_o', 'source_h', 'source_r', 'source_s', 'source_t',
       'source_u', 'source_p', 'source_x', 'source_w', 'Web order',
       'Gender=male', 'Address_is_res']
non_binary_cols = [col for col in X_features.columns if col not in binary_cols]

In [58]:
x_scaler = StandardScaler()
# y_scaler = StandardScaler()

X_CV_scaled = X_CV.copy()
X_CV_scaled[non_binary_cols] = x_scaler.fit_transform(X_CV_scaled[non_binary_cols])

X_test_scaled = X_test.copy()
X_test_scaled[non_binary_cols] = x_scaler.transform(X_test[non_binary_cols])

# y_CV_scaled = y_scaler.fit_transform(y_CV.values.reshape(-1, 1)).flatten()
# y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1)).flatten()

In [59]:
y_feature.describe()

,Spending
count,2000.000000
mean,102.560745
std,186.749816
min,0.000000
25%,0.000000
50%,1.855000
75%,152.532500
max,1500.060000


In [60]:
X_CV_scaled[non_binary_cols]

,Freq,last_update_days_ago,1st_update_days_ago
232,-0.295438,-0.400055,-0.678144
1211,-1.035191,0.844703,0.636131
1352,-0.295438,-1.323273,-1.652922
1721,0.444314,-0.170132,-0.415847
1492,-0.295438,-1.038732,-1.352489
...,...,...,...
399,0.444314,-1.495055,-1.491079
1418,-0.295438,-1.115373,-1.433411
1606,-0.295438,1.042913,0.845411
1089,-0.295438,0.779514,0.567302


In [64]:
# Define regression models
models = {
    "LinearRegression": LinearRegression(),
    "KNN": KNeighborsRegressor(),
    "DecisionTree": DecisionTreeRegressor(),
    "SVR": SVR(),
    "NeuralNet": MLPRegressor(
        max_iter=500,
        early_stopping=True,
        n_iter_no_change=10,
        validation_fraction=0.1,
        random_state=42
    ),
    "RandomForest": RandomForestRegressor(),
    "XGBoost": XGBRegressor(verbosity=0, random_state=42)
}

# Define hyperparameter grids
param_grids = {
    "LinearRegression": {},
    "KNN": {
        "model__n_neighbors": [3, 5, 7],
        "model__weights": ["uniform", "distance"]
    },
    "DecisionTree": {
        "model__max_depth": [4, 6, 10, None],
        "model__min_samples_split": [2, 5, 10]
    },
    "SVR": {
        "model__C": [0.1, 1, 10],
        "model__kernel": ["linear", "rbf"]
    },
    "NeuralNet": {
        "model__hidden_layer_sizes": [(50,), (100,), (50, 50)],
        "model__alpha": [0.0001, 0.001, 0.01],
        "model__learning_rate_init": [0.0005, 0.001, 0.005],
        "model__solver": ['adam'],
        "model__activation": ['relu', 'tanh']
    },
    "RandomForest": {
        "model__n_estimators": [50, 100],
        "model__max_depth": [5, 10, None]
    },
    "XGBoost": {
        "model__n_estimators": [50, 100],
        "model__max_depth": [3, 5],
        "model__learning_rate": [0.05, 0.1],
        "model__subsample": [0.8],
        "model__colsample_bytree": [0.8],
        "model__reg_alpha": [0, 0.1],
        "model__reg_lambda": [1]
    }
}

# Define scoring function (RMSE)
rmse_scorer = make_scorer(mean_squared_error, squared=False)

# Define outer and inner CV
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Store results
results = {}

# Nested cross-validation
for model_name, model in models.items():
    print(f"Running Nested CV for {model_name}...")
    nested_rmse_scores = []

    for train_idx, valid_idx in outer_cv.split(X_CV_scaled, y_CV):
        X_train, X_valid = X_CV_scaled.iloc[train_idx], X_CV_scaled.iloc[valid_idx]
        y_train, y_valid = y_CV.iloc[train_idx], y_CV.iloc[valid_idx]

        pipeline = Pipeline([("scaler", StandardScaler()), ("model", model)])

        if model_name in ["SVR", "NeuralNet"]:
            pipeline = TransformedTargetRegressor(
                regressor=pipeline,
                transformer=StandardScaler()  # scales y
            )

        param_grid = param_grids[model_name]
        if model_name in ["SVR", "NeuralNet"]:
            param_grid = {
                f"regressor__{k}": v for k, v in param_grid.items()
            }

        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            scoring=rmse_scorer,
            cv=inner_cv,
            refit=True,
            n_jobs=-1
        )

        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_

        y_pred = best_model.predict(X_valid)
        rmse = np.sqrt(np.mean((y_valid - y_pred) ** 2))
        nested_rmse_scores.append(rmse)

    mean_rmse = np.mean(nested_rmse_scores)
    std_rmse = np.std(nested_rmse_scores)

    results[model_name] = {"mean_rmse": mean_rmse, "std_rmse": std_rmse}
    print(f"{model_name} - Mean RMSE: {mean_rmse:.4f}, Std: {std_rmse:.4f}")

# import ace_tools as tools; tools.display_dataframe_to_user(name="Nested CV RMSE Results", dataframe=pd.DataFrame(results).T)

Running Nested CV for LinearRegression...
LinearRegression - Mean RMSE: 128.1178, Std: 14.4484
Running Nested CV for KNN...
KNN - Mean RMSE: 147.9721, Std: 15.2731
Running Nested CV for DecisionTree...
DecisionTree - Mean RMSE: 140.0640, Std: 21.0021
Running Nested CV for SVR...
SVR - Mean RMSE: 132.5544, Std: 16.1471
Running Nested CV for NeuralNet...
NeuralNet - Mean RMSE: 129.5624, Std: 13.7240
Running Nested CV for RandomForest...
RandomForest - Mean RMSE: 131.0460, Std: 16.4863
Running Nested CV for XGBoost...
XGBoost - Mean RMSE: 129.9684, Std: 17.8349


Above performance showed that Linear Regression, Neural Networks, and XGBoost had the lowest RMSE values.

Although Linear Regression had the slightly lowest RMSE, it assumes linearity and may underperform when faced with complex feature interactions. Given that the Neural Net performs nearly as well but offers more flexibility as shown by lower Standard deviation, it makes a stronger case as the more robust and generalizable choice for predicting Spending. Below, I proceed to retraining on entire training dataset using keras

In [67]:
import keras_tuner as kt
from keras.metrics import RootMeanSquaredError

In [68]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(
        units=hp.Choice('units_1', [32, 64, 128]),
        activation=hp.Choice('activation_1', ['relu']),
        input_shape=(X_CV.shape[1],),
        kernel_regularizer=l2(hp.Float('l2_1', 1e-5, 1e-2, sampling='log'))
    ))
    if hp.Boolean('add_second_layer'):
        model.add(Dense(
            units=hp.Choice('units_2', [32, 64]),
            activation=hp.Choice('activation_2', ['relu']),
            kernel_regularizer=l2(hp.Float('l2_2', 1e-5, 1e-2, sampling='log'))
        ))
    model.add(Dense(1))

    model.compile(
        optimizer=Adam(hp.Float('lr', 1e-4, 1e-2, sampling='log')),
        loss='mse',
        metrics=[RootMeanSquaredError(name="root_mean_squared_error")]
    )
    return model

In [69]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_root_mean_squared_error',
    max_trials=10,
    executions_per_trial=1,
    directory='keras_tuner_dir',
    project_name='neural_net_regression',
    overwrite=True,
)

tuner.search_space_summary()

# Normalize target
target_scaler = StandardScaler()
y_CV_scaled = target_scaler.fit_transform(y_CV.values.reshape(-1, 1)).ravel()

tuner.search(X_CV_scaled, y_CV_scaled, validation_split=0.2, epochs=100, batch_size=32, verbose=1)


Trial 10 Complete [00h 00m 31s]
val_root_mean_squared_error: 0.8063823580741882

Best val_root_mean_squared_error So Far: 0.7904695868492126
Total elapsed time: 00h 05m 09s


In [71]:
best_hp = tuner.get_best_hyperparameters(1)[0]
model = tuner.hypermodel.build(best_hp)

history = model.fit(X_CV_scaled, y_CV_scaled, epochs=100, batch_size=32, verbose=1)

y_pred_scaled = model.predict(X_test_scaled)
y_pred = target_scaler.inverse_transform(y_pred_scaled)


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.7258 - root_mean_squared_error: 0.8459
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5770 - root_mean_squared_error: 0.7537
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4082 - root_mean_squared_error: 0.6339
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3333 - root_mean_squared_error: 0.5733
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3975 - root_mean_squared_error: 0.6277
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4879 - root_mean_squared_error: 0.6932
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3808 - root_mean_squared_error: 0.6146
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3265 - root_mean_squared_error: 0.5670
Epoch 9/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3447 - root_mean_squared_error: 0.5828
Epoch 10/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3861 - root_mean_squared_error: 0.6154
Epoch 11/10

In [72]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Final RMSE on Test Set: {rmse:.4f}")

Final RMSE on Test Set: 149.3733


The final RMSE of 149.3733 on the test set confirms that the Neural Network model trained on the full dataset generalizes well and performs comparably to its cross-validated estimate

### Q1) (B)

In [76]:
df = pd.read_excel(excel_file_path)
df.drop(['sequence_number'], axis=1, inplace=True)

In [77]:
df = df[df['Purchase'] == 1]
X_features = df.drop(columns=['Purchase', 'Spending'], axis=1)
y_feature = df['Spending']

In [78]:
X_CV, X_test, y_CV, y_test = train_test_split(
    X_features, y_feature, test_size=0.20, random_state=60
)

In [79]:
x_scaler = StandardScaler()

X_CV_scaled = X_CV.copy()
X_CV_scaled[non_binary_cols] = x_scaler.fit_transform(X_CV_scaled[non_binary_cols])

X_test_scaled = X_test.copy()
X_test_scaled[non_binary_cols] = x_scaler.transform(X_test[non_binary_cols])

In [80]:
# Define regression models
models = {
    "LinearRegression": LinearRegression(),
    "KNN": KNeighborsRegressor(),
    "DecisionTree": DecisionTreeRegressor(),
    "SVR": SVR(),
    "NeuralNet": MLPRegressor(
        max_iter=500,
        early_stopping=True,
        n_iter_no_change=10,
        validation_fraction=0.1,
        random_state=42
    ),
    "RandomForest": RandomForestRegressor(),
    "XGBoost": XGBRegressor(verbosity=0, random_state=42)
}

# Define hyperparameter grids
param_grids = {
    "LinearRegression": {},
    "KNN": {
        "model__n_neighbors": [3, 5, 7],
        "model__weights": ["uniform", "distance"]
    },
    "DecisionTree": {
        "model__max_depth": [4, 6, 10, None],
        "model__min_samples_split": [2, 5, 10]
    },
    "SVR": {
        "model__C": [0.1, 1, 10],
        "model__kernel": ["linear", "rbf"]
    },
    "NeuralNet": {
        "model__hidden_layer_sizes": [(50,), (100,), (50, 50)],
        "model__alpha": [0.0001, 0.001, 0.01],
        "model__learning_rate_init": [0.0005, 0.001, 0.005],
        "model__solver": ['adam'],
        "model__activation": ['relu', 'tanh']
    },
    "RandomForest": {
        "model__n_estimators": [50, 100],
        "model__max_depth": [5, 10, None]
    },
    "XGBoost": {
        "model__n_estimators": [50, 100],
        "model__max_depth": [3, 5],
        "model__learning_rate": [0.05, 0.1],
        "model__subsample": [0.8],
        "model__colsample_bytree": [0.8],
        "model__reg_alpha": [0, 0.1],
        "model__reg_lambda": [1]
    }
}

# Define scoring function (RMSE)
rmse_scorer = make_scorer(mean_squared_error, squared=False)

# Define outer and inner CV
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Store results
results = {}

# Nested cross-validation
for model_name, model in models.items():
    print(f"Running Nested CV for {model_name}...")
    nested_rmse_scores = []

    for train_idx, valid_idx in outer_cv.split(X_CV_scaled, y_CV):
        X_train, X_valid = X_CV_scaled.iloc[train_idx], X_CV_scaled.iloc[valid_idx]
        y_train, y_valid = y_CV.iloc[train_idx], y_CV.iloc[valid_idx]

        pipeline = Pipeline([("scaler", StandardScaler()), ("model", model)])

        if model_name in ["SVR", "NeuralNet"]:
            pipeline = TransformedTargetRegressor(
                regressor=pipeline,
                transformer=StandardScaler()  # scales y
            )

        param_grid = param_grids[model_name]
        if model_name in ["SVR", "NeuralNet"]:
            param_grid = {
                f"regressor__{k}": v for k, v in param_grid.items()
            }

        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            scoring=rmse_scorer,
            cv=inner_cv,
            refit=True,
            n_jobs=-1
        )

        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_

        y_pred = best_model.predict(X_valid)
        rmse = np.sqrt(np.mean((y_valid - y_pred) ** 2))
        nested_rmse_scores.append(rmse)

    mean_rmse = np.mean(nested_rmse_scores)
    std_rmse = np.std(nested_rmse_scores)

    results[model_name] = {"mean_rmse": mean_rmse, "std_rmse": std_rmse}
    print(f"{model_name} - Mean RMSE: {mean_rmse:.4f}, Std: {std_rmse:.4f}")

Running Nested CV for LinearRegression...
LinearRegression - Mean RMSE: 168.8176, Std: 27.2881
Running Nested CV for KNN...
KNN - Mean RMSE: 209.1332, Std: 12.7428
Running Nested CV for DecisionTree...
DecisionTree - Mean RMSE: 192.8781, Std: 31.8473
Running Nested CV for SVR...
SVR - Mean RMSE: 174.6933, Std: 24.5330
Running Nested CV for NeuralNet...
NeuralNet - Mean RMSE: 172.3232, Std: 24.5004
Running Nested CV for RandomForest...
RandomForest - Mean RMSE: 173.1391, Std: 25.6382
Running Nested CV for XGBoost...
XGBoost - Mean RMSE: 166.9481, Std: 24.2154


XGBoost will be chosen because it consistently achieved the lowest RMSE (166.95) in nested cross-validation across all models, indicating superior predictive accuracy on the restricted dataset of actual purchasers.

In [82]:
xgb_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", XGBRegressor(verbosity=0, random_state=42))
])

xgb_param_grid = {
    "model__n_estimators": [50, 100],
    "model__max_depth": [3, 5],
    "model__learning_rate": [0.01, 0.1],
    "model__subsample": [0.8, 1.0],
    "model__colsample_bytree": [0.8, 1.0]
}

# Define GridSearchCV
xgb_grid = GridSearchCV(
    estimator=xgb_pipeline,
    param_grid=xgb_param_grid,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=1
)

xgb_grid.fit(X_CV_scaled, y_CV)

print("Best Hyperparameters:", xgb_grid.best_params_)

y_pred = xgb_grid.predict(X_test_scaled)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Test Set RMSE with Tuned XGBoost: {rmse:.4f}")

Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best Hyperparameters: {'model__colsample_bytree': 1.0, 'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 100, 'model__subsample': 1.0}
Test Set RMSE with Tuned XGBoost: 125.3202


### Q1) (C)

Task (a) involved building predictive models on the entire dataset, while Task (b) focused only on a subset of users who made a purchase (Purchase == 1). Across all predictive modeling techniques (Linear Regression, KNN, Decision Tree, SVR, Neural Network, Random Forest, and XGBoost), the models built for Task (b) exhibited better predictive performance on the test set, as reflected by lower RMSE values.

This may seem counterintuitive at first, since Task (b) used less data, but there are two key reasons for this performance improvement:

 - Reduced Variability and Noise:
The subset in Task (b) excludes non-purchasers (many of whom have very low or zero spending). This makes the spending behavior more consistent and easier to model, reducing target variability and prediction noise.

 - More Predictable Patterns:
By focusing only on actual purchasers, the models could learn from clearer, more relevant patterns — as opposed to needing to generalize across both purchasers and non-purchasers in Task (a).

 - Task (a) essentially combines a classification (who will purchase?) and regression (how much will they spend?) problem. Task (b), on the other hand, is a pure regression problem focused on estimating spending among purchasers, which aligns better with the chosen regression algorithms.

Therefore, although Task (b) used fewer data points, it allowed for sharper signal and less noise, which improved model accuracy across the board. The best test RMSE was achieved using a tuned XGBoost model trained on the Task (b) subset.